# Заполним пропуски

Пропуски есть в near_metro и class

# Near metro
## Заполним через api yandex "поиск по организациям"

In [1]:
import json
import geopy.distance as distance
import pandas as pd
import requests

In [2]:
data = pd.read_csv('../result.csv')

In [3]:
def get_near_metro(lon, lat, api_key):
    req_text = 'https://search-maps.yandex.ru/v1/?text=метро&ll=' + str(lon) + "," + str(lat) + '&lang=ru_RU&results=1&apikey=' + api_key
    ans = json.loads(requests.get(req_text).text)
    try:
        metro_lon, metro_lat = ans['features'][0]['geometry']['coordinates']
        result = distance.vincenty((lon, lat), (metro_lon, metro_lat)).m
    except:
        result = np.nan
    return result

In [4]:
null_metro = data[data['near_metro'].isnull()]

In [5]:
null_metro.head(2)

,Unnamed: 0,size,class,address,price,pledge,descr,metros,near_metro,url,parking,internet,meal,security,comfort,lat,lon,distance
39,1002926616,57.0,NaN,"Петровско-Разумовская, Лихоборская наб 7",685,0,Сдаются офис склад на территории бывшего про...,NaN,NaN,https://www.avito.ru/moskva/kommercheskaya_ned...,1,0,0,0,0,55.854411,37.529070,0.20401
56,1690633446,39.0,NaN,"Автозаводская, Кожуховский 2-й проезд 29к2с...",1085,0,ДО КОНЦА 2018 ГОДА МЕСЯЦ АРЕНДЫ ЭТОГО ПОМЕЩЕ...,NaN,NaN,https://www.avito.ru/moskva/kommercheskaya_ned...,1,0,0,1,1,55.698391,37.658248,0.09784


In [6]:
with open('../apies/yandex_organization_search_api.txt') as f:
    apikey = f.readline()

Через API можно отправлять только 500 запросов, раз в сутки

In [7]:
for index, row in null_metro.iterrows():
    null_metro.at[index, 'near_metro'] = get_near_metro(row['lon'], row['lat'], apikey)

In [10]:
data[data['near_metro'].isnull()] = null_metro
data.to_csv('got_near_metro.csv')

# Class

In [11]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score

In [63]:
data = pd.read_csv('got_near_metro.csv')

In [64]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3155 entries, 0 to 3154
Data columns (total 19 columns):
Unnamed: 0      3155 non-null int64
Unnamed: 0.1    3155 non-null int64
size            3155 non-null float64
class           1954 non-null object
address         3155 non-null object
price           3155 non-null int64
pledge          3155 non-null int64
descr           3155 non-null object
metros          2999 non-null object
near_metro      3155 non-null float64
url             3155 non-null object
parking         3155 non-null int64
internet        3155 non-null int64
meal            3155 non-null int64
security        3155 non-null int64
comfort         3155 non-null int64
lat             3155 non-null float64
lon             3155 non-null float64
distance        3155 non-null float64
dtypes: float64(5), int64(9), object(5)
memory usage: 468.4+ KB


In [65]:
data.head(5)

,Unnamed: 0,Unnamed: 0.1,size,class,address,price,pledge,descr,metros,near_metro,url,parking,internet,meal,security,comfort,lat,lon,distance
0,0,1061223879,28.8,NaN,"Москва, м. Марксистская, улица Александра Сол...",1354,39000,Помещение на 2 этаже административного здания....,Марксистская (700 м) . Таганская (800 м) . ...,700.0,https://www.avito.ru/moskva/kommercheskaya_ned...,0,0,0,0,1,55.745093,37.664815,0.09581
1,1,1335488313,588.2,NaN,"Москва, м. Марксистская, улица Александра Сол...",850,500000,Блок комнат на 4 этаже административного здани...,Марксистская (700 м) . Таганская (800 м) . ...,700.0,https://www.avito.ru/moskva/kommercheskaya_ned...,1,0,0,0,1,55.745093,37.664815,0.09581
2,2,1617672126,45.0,NaN,"Москва, м. Шоссе Энтузиастов, Электродный про...",1111,50000,Кабинет на 1 этаже административного здания. ...,Шоссе Энтузиастов (200 м) . Соколиная гора ...,200.0,https://www.avito.ru/moskva/kommercheskaya_ned...,1,0,0,1,0,55.759659,37.752850,0.27184
3,3,1585544645,100.0,NaN,"Москва, м. Бульвар Дмитрия Донского, Старокач...",1700,170000,Сдаётся офис 100 м2 в элитном ЖК «Синяя Птица»...,Бульвар Дмитрия Донского (700 м) . Улица Ст...,700.0,https://www.avito.ru/moskva/kommercheskaya_ned...,1,0,0,1,0,55.569026,37.588647,0.19060
4,4,1567297779,190.0,NaN,"Москва, м. Новогиреево, Федеративный проспект...",789,150000,1 этаж жилого дома. Отдельный вход. 4 кабинета...,Новогиреево (1.2 км) . Перово (1.2 км) . Ш...,1200.0,https://www.avito.ru/moskva/kommercheskaya_ned...,0,1,0,0,0,55.757496,37.802086,0.37023


In [66]:
data = data.drop(['Unnamed: 0', 'Unnamed: 0.1', 'address', 'descr', 'metros', 'url'], axis=1)
data.head(5)

,size,class,price,pledge,near_metro,parking,internet,meal,security,comfort,lat,lon,distance
0,28.8,NaN,1354,39000,700.0,0,0,0,0,1,55.745093,37.664815,0.09581
1,588.2,NaN,850,500000,700.0,1,0,0,0,1,55.745093,37.664815,0.09581
2,45.0,NaN,1111,50000,200.0,1,0,0,1,0,55.759659,37.752850,0.27184
3,100.0,NaN,1700,170000,700.0,1,0,0,1,0,55.569026,37.588647,0.19060
4,190.0,NaN,789,150000,1200.0,0,1,0,0,0,55.757496,37.802086,0.37023


Цена является целевой переменной, поэтому использовать ее для заполнения пропусков нельзя.

Нужно разделить на обучающую и целевую выборку.

In [67]:
null_classes = data[data['class'].isnull()].drop(['price'], axis=1)
not_null_classes = data[~data['class'].isnull()].drop(['price'], axis=1)

Заполним Nan-ы knn - ом.

In [68]:
null_classes.head(5)

,size,class,pledge,near_metro,parking,internet,meal,security,comfort,lat,lon,distance
0,28.8,NaN,39000,700.0,0,0,0,0,1,55.745093,37.664815,0.09581
1,588.2,NaN,500000,700.0,1,0,0,0,1,55.745093,37.664815,0.09581
2,45.0,NaN,50000,200.0,1,0,0,1,0,55.759659,37.752850,0.27184
3,100.0,NaN,170000,700.0,1,0,0,1,0,55.569026,37.588647,0.19060
4,190.0,NaN,150000,1200.0,0,1,0,0,0,55.757496,37.802086,0.37023


In [69]:
X_data = not_null_classes.drop(['class'], axis=1)
y_data = not_null_classes['class'].copy()
scaler = StandardScaler()
X_data = scaler.fit_transform(X_data)
X_data

array([[ -2.94019133e-01,  -2.30754993e-01,  -8.50385949e-01, ...,
         -1.70871113e+00,  -1.94841224e+00,   2.01179132e+00],
       [ -7.75937377e-02,   7.74952482e-02,  -5.10192330e-01, ...,
          4.21492520e-01,   6.79631533e-01,  -3.71951819e-01],
       [ -3.15468436e-01,  -2.03499125e-01,   9.80981614e-05, ...,
         -3.84584458e-01,  -1.59500452e+00,   1.26580109e+00],
       ..., 
       [ -3.01555375e-01,  -2.56064014e-01,  -8.50385949e-01, ...,
         -6.04924269e-01,   4.27774084e-01,  -7.10092312e-01],
       [  7.02325369e-02,  -2.95000968e-01,   5.10388527e-01, ...,
         -2.01991615e+00,  -2.90314959e-01,  -1.08769485e-01],
       [  1.53110396e+00,  -2.44382927e-01,  -3.40095521e-01, ...,
          5.83403852e-01,   6.38529682e-01,  -4.12746002e-01]])

Можно написать свой скелер, чтобы он не скейлил категориальные признаки.

In [70]:
clf = KNeighborsClassifier()
scores = cross_val_score(clf, X_data, y_data, cv=15)
print(scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

[ 0.71428571  0.68421053  0.6969697   0.75        0.69465649  0.63358779
  0.70769231  0.75968992  0.6744186   0.6744186   0.70542636  0.73643411
  0.74418605  0.72093023  0.81395349]
Accuracy: 0.71 (+/- 0.08)


In [71]:
clf.fit(X_data, y_data)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [72]:
Target_data = null_classes.drop(['class'], axis=1)
Target_data = pd.DataFrame(scaler.transform(Target_data))

In [73]:
Target_data.head(5)

,0,1,2,3,4,5,6,7,8,9,10
0,-0.299237,-0.219074,-0.169999,-1.030137,-0.789255,-0.690613,-1.010288,-0.802013,-0.049097,0.560967,-0.593695
1,0.781731,0.678423,-0.169999,0.970745,-0.789255,-0.690613,-1.010288,-0.802013,-0.049097,0.560967,-0.593695
2,-0.267932,-0.197659,-1.020483,0.970745,-0.789255,-0.690613,0.989816,-1.227749,0.171273,1.529231,1.027298
3,-0.161652,0.035963,-0.169999,0.970745,-0.789255,-0.690613,0.989816,-1.227749,-2.712828,-0.276776,0.279190
4,0.012261,-0.002974,0.680485,-1.030137,1.267017,-0.690613,-1.010288,-1.227749,0.138549,2.070759,1.933334


In [74]:
#Target_data['class'] = clf.predict(Target_data)
classes = clf.predict(Target_data)

In [75]:
classes

array(['b', 'b', 'b', ..., 'b', 'b', 'b'], dtype=object)

In [76]:
data.loc[data['class'].isnull(), 'class'] = classes

In [77]:
data.head(5)

,size,class,price,pledge,near_metro,parking,internet,meal,security,comfort,lat,lon,distance
0,28.8,b,1354,39000,700.0,0,0,0,0,1,55.745093,37.664815,0.09581
1,588.2,b,850,500000,700.0,1,0,0,0,1,55.745093,37.664815,0.09581
2,45.0,b,1111,50000,200.0,1,0,0,1,0,55.759659,37.752850,0.27184
3,100.0,b,1700,170000,700.0,1,0,0,1,0,55.569026,37.588647,0.19060
4,190.0,c,789,150000,1200.0,0,1,0,0,0,55.757496,37.802086,0.37023


In [78]:
data.to_csv('data_with_classes_and_metros.csv')